In [1]:
from flask import Flask, request, jsonify, render_template
from tensorflow.keras.models import load_model
import numpy as np
from PIL import Image
import sqlite3
import os
from datetime import datetime

app = Flask(__name__)

# Load Model
model = load_model("eye_disease_model.h5")

# Database Setup
def init_db():
    conn = sqlite3.connect("patients.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            patient_id TEXT,
            name TEXT,
            uploaded_image TEXT,
            disease TEXT,
            percentage REAL,
            date TIMESTAMP
        )
    """)
    conn.commit()
    conn.close()

init_db()

categories = ["Glaucoma", "Cataract", "Diabetic Retinopathy", "Normal"]

@app.route('/')
def index():
    return render_template("index.html")

@app.route('/predict', methods=['POST'])
def predict():
    # Patient Details
    patient_id = request.form['patient_id']
    name = request.form['name']
    file = request.files['image']

    # Save Uploaded Image
    upload_path = os.path.join("uploads", file.filename)
    file.save(upload_path)

    # Process Image
    img = Image.open(upload_path).resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict Disease
    prediction = model.predict(img_array)[0]
    max_idx = np.argmax(prediction)
    disease = categories[max_idx]
    percentage = round(prediction[max_idx] * 100, 2)

    # Save to Database
    conn = sqlite3.connect("patients.db")
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO history (patient_id, name, uploaded_image, disease, percentage, date)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (patient_id, name, upload_path, disease, percentage, datetime.now()))
    conn.commit()

    # Fetch Previous Records
    cursor.execute("SELECT * FROM history WHERE patient_id = ?", (patient_id,))
    history = cursor.fetchall()
    conn.close()

    # Suggestions
    suggestions = "Consult a doctor immediately." if percentage > 50 else "Consider routine check-up."
    if disease == "Normal":
        suggestions = "No disease detected."

    return render_template("result.html", patient_id=patient_id, name=name, disease=disease,
                           percentage=percentage, suggestions=suggestions, history=history)

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\Manu\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
